In [1]:
import dspy
import dspy
from dspy.evaluate import SemanticF1
from dspy import settings
import os

/Users/Salim/Library/Caches/pypoetry/virtualenvs/dspy-toy-SOZM3orb-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
api_key = os.environ.get('OPENAI_API_KEY')

In [14]:
lm = dspy.LM('openai/gpt-4.1-nano', api_key=api_key)
dspy.configure(lm=lm)

In [15]:
mom_test_examples = [
  {
    "business_proposal": "One of my companies had some legal ambiguities around content ownership. It was okay in theory, but lacked strong precedents. I was always a bit nervous we would get 'found out.' Their execs were excited, their creatives were thrilled. We had even brought the stubborn tech team around to our side.",
    "question": "Can I speak with your legal team to understand if they’re comfortable with the content ownership terms?"
  },
  {
    "business_proposal": "We suspect that teachers from the poorest schools are completely overloaded, and that our tools would save them time so they could better educate their students.",
    "question": "Do you have a budget allocated for tools like this?"
  },
  {
    "business_proposal": "Bars will pay, but only if you can amass a huge audience of consumers. The founders talk to consumers and ask if they would use an app which always pointed them to booming parties with cheap booze.",
    "question": "How would we reach the critical mass of users needed to make this appealing to bars?"
  },
  {
    "business_proposal": "You’ve got $50k to start your business and you want to know whether your idea is any good before committing all your resources.",
    "question": "What’s the most likely reason this business could fail?"
  },
  {
    "business_proposal": "You’re building a solution for a customer segment that includes 'students' or 'sales organizations,' thinking you’re focused.",
    "question": "Am I talking to a consistent customer segment or are these conversations spread across too many types?"
  }
]


In [16]:
synthetic_examples = [
  {
    "business_proposal": "We’re building a marketplace for local artisans to sell their crafts directly to consumers, cutting out the middleman.",
    "question": "Have you ever tried selling your products online, and what stopped you if you haven’t?"
  },
  {
    "business_proposal": "We want to automate nutrition planning for athletes with an AI coach that updates meal plans daily based on performance.",
    "question": "How do you currently plan your meals and how often do you actually adjust them based on training feedback?"
  },
  {
    "business_proposal": "Our product helps landlords track maintenance requests and rent payments in one dashboard.",
    "question": "What are the biggest problems in your rental workflow right now, and do any of them actually cost you money or tenants?"
  },
  {
    "business_proposal": "We’re launching a subscription box of STEM projects for kids aged 8–12 to foster early interest in engineering.",
    "question": "When was the last time you bought something educational for your kids that wasn’t required by school?"
  },
  {
    "business_proposal": "We're building a tool to help wedding planners coordinate vendors, guests, and schedules in one place.",
    "question": "What’s the worst part of planning a wedding, and how do you currently manage that part?"
  },
  {
    "business_proposal": "We want to help freelancers manage their taxes and automatically save for quarterly payments.",
    "question": "Have you ever missed a tax payment or been hit with a fine? What did you do about it?"
  },
  {
    "business_proposal": "We’re developing a social platform for book lovers to find and join local book clubs.",
    "question": "How do you currently find new people to read or discuss books with, and do you feel like that’s a gap in your life?"
  },
  {
    "business_proposal": "We’re building software to help nonprofit boards manage compliance and governance more easily.",
    "question": "Has governance or compliance ever caused real delays or risks for your nonprofit?"
  },
  {
    "business_proposal": "We want to make a budgeting app for couples to track and manage shared expenses transparently.",
    "question": "What’s the last disagreement you had with your partner about money, and how did you resolve it?"
  },
  {
    "business_proposal": "We’re building an AI tool to help content marketers repurpose long-form blogs into social media posts.",
    "question": "What’s stopping you from turning your best content into social posts today?"
  }
]

In [17]:
trainset = [
    dspy.Example(business_proposal=x.get('business_proposal'), label=x.get('question')).with_inputs("business_proposal")
    for x in mom_test_examples + synthetic_examples
]

In [18]:
trainset[:2]

[Example({'business_proposal': "One of my companies had some legal ambiguities around content ownership. It was okay in theory, but lacked strong precedents. I was always a bit nervous we would get 'found out.' Their execs were excited, their creatives were thrilled. We had even brought the stubborn tech team around to our side.", 'label': 'Can I speak with your legal team to understand if they’re comfortable with the content ownership terms?'}) (input_keys={'business_proposal'}),
 Example({'business_proposal': 'We suspect that teachers from the poorest schools are completely overloaded, and that our tools would save them time so they could better educate their students.', 'label': 'Do you have a budget allocated for tools like this?'}) (input_keys={'business_proposal'})]

In [23]:
import dspy
from dspy.evaluate import SemanticF1
from dspy import settings

settings.use_structured_output = False   # stop adding response_format
settings.rationale_mode        = "none"  # stop adding rationale_type

dspy.configure(lm=dspy.LM('openai/gpt-3.5-turbo-0125'))

sig = dspy.ChainOfThought("business_proposal -> question: str") 
metric = SemanticF1(threshold=0.75)
optim = dspy.BootstrapFewShot(metric=metric)
scary_agent = optim.compile(sig, trainset=trainset)

  0%|                                                                                                                                         | 0/15 [00:00<?, ?it/s]2025/05/22 22:26:33 ERROR dspy.teleprompt.bootstrap: Failed to run or to evaluate example Example({'business_proposal': "One of my companies had some legal ambiguities around content ownership. It was okay in theory, but lacked strong precedents. I was always a bit nervous we would get 'found out.' Their execs were excited, their creatives were thrilled. We had even brought the stubborn tech team around to our side.", 'label': 'Can I speak with your legal team to understand if they’re comfortable with the content ownership terms?'}) (input_keys={'business_proposal'}) with module.predict = Predict(StringSignature(question, ground_truth, system_response -> reasoning, recall, precision
    instructions="Compare a system's response to the ground truth to compute its recall and precision.\nIf asked to reason, enumerate key ideas

AttributeError: 'Example' object has no attribute 'question'

In [11]:
sig(business_proposal="One of my companies had some legal ambiguities around content ownership. It was okay in theory, but lacked strong precedents. I was always a bit nervous we would get 'found out.' Their execs were excited, their creatives were thrilled. We had even brought the stubborn tech team around to our side.")

Prediction(
    reasoning='The legal ambiguities surrounding content ownership in the company\'s operations created a sense of unease and uncertainty. Despite the excitement from the executives and creatives, the lack of strong precedents and the potential risk of being "found out" remained a lingering concern.',
    scary_question="What potential consequences could arise if the legal ambiguities around content ownership are exposed, and how could this impact the company's reputation and future operations?"
)

In [ ]:
metric = SemanticF1(threshold=0.75)
optim = dspy.BootstrapFinetune(metric=metric, num_threads=1)
scary_agent = optim.compile(ask_scary, trainset=trainset)

In [ ]:
proposal = """
We’re building an app that lets bars live‑stream trivia so patrons
in different locations can compete in real time.
"""
print(scary_agent(business_proposal=proposal).scary_question)